In [1]:
import numpy as np
import pandas as pd
import tifffile as tiff
import zarr
import napari
import random
import dask.array as da
from dask.cache import Cache
import zarr
import os
import numpy as np
from pathlib import Path
from magicgui import magicgui
import ast
import distinctipy 
from matplotlib.colors import to_hex

In [2]:
viewer = napari.Viewer()

In [3]:
@magicgui(call_button='Open image')
def open_image(impath = Path(), channel_list=Path()):
    image = tiff.TiffFile(impath, is_ome = False)
    z = zarr.open(image.aszarr(), mode='r')
    n_levels = len(image.series[0].levels) # pyramid

    # If and if not pyramids are available
    if n_levels > 1:
        pyramid = [da.from_zarr(z[i]) for i in range(n_levels)]
        multiscale = True
    else:
        pyramid = da.from_zarr(z)
        multiscale = False 
        
    if channel_list == '.':
        viewer.add_image(pyramid, multiscale=multiscale, channel_axis=0, visible=False, contrast_limits=(0,65535))
    else:
        list_df = pd.read_excel(channel_list)
        clist = list(list_df.loc[:, 'Channel_name'])
        viewer.add_image(pyramid, multiscale=multiscale, channel_axis=0, visible=False, contrast_limits=(0,65535), name=clist)

viewer.window.add_dock_widget(open_image)

In [4]:
RED = (1, 0, 0)
GREEN = (0, 1, 0)
BLUE = (0, 0, 1)
WHITE = (1, 1, 1)
BLACK = (0, 0, 0)
MAGENTA = (1, 0, 1)
YELLOW = (1, 1, 0)
CYAN = (0, 1, 1)

excluded_colors = [RED, GREEN, BLUE, BLACK, WHITE, CYAN, MAGENTA]

In [5]:
def filter_and_transform(arr, selected_ids):
    filtered_arr = np.where(np.isin(arr, selected_ids), arr, 0)
    return np.where(filtered_arr != 0, 1, 0)

In [11]:
@magicgui(call_button = 'Show celltype masks')
def show_celltypes1(mask_path = Path(), labels = Path()):

    c_table = pd.read_csv(labels)
    cts = c_table['final_label'].unique()
    ids = c_table['ID'].unique()
    filtered_cellIDs = c_table.loc[c_table['final_label'] == cts[0], 'ID'].values 

    channel=0
    image = tiff.TiffFile(mask_path)
    img = image.pages[channel].asarray()
    pyramid = [img[::4**i, ::4**i] for i in range(4) ]
    pyramid = [da.from_array(z) for z in pyramid]

    colors = distinctipy.get_colors(len(cts), pastel_factor=0.6, exclude_colors = excluded_colors)
    hex_colors = [to_hex(color) for color in colors]
    
    for celltype in range(len(cts)):
        color = list(colors[celltype])
        color.append(1)
        new_cmap = napari.utils.colormaps.Colormap(colors = np.array([[0.0, 0.0, 0.0, 0.0],color]), name = hex_colors[celltype])
        print(cts[celltype])
        filtered_cellIDs = c_table.loc[c_table['final_label'] == cts[celltype], 'ID'].values 
        filtered_pyramid = [da.map_blocks(filter_and_transform, layer, filtered_cellIDs, dtype=np.uint8) for layer in pyramid]
        viewer.add_image(filtered_pyramid, colormap= new_cmap, name= cts[celltype] + '_mask', blending = 'translucent', contrast_limits=(0,1))  

viewer.window.add_dock_widget(show_celltypes1)